In [0]:
# 连接上谷歌云盘
from google.colab import drive 
drive.mount('/content/drive')
!pwd

import os
os.chdir('drive/My Drive/5003project')
!pwd

In [0]:
import pandas as pd
import numpy as np
import time

In [0]:
rent_file = 'renttherunway_final_data.json'
rent_df = pd.read_json(rent_file, lines=True)
rent_df.head()

,age,body type,bust size,category,fit,height,item_id,rating,rented for,review_date,review_summary,review_text,size,user_id,weight
0,28.0,hourglass,34d,romper,fit,"5' 8""",2260466,10.0,vacation,"April 20, 2016",So many compliments!,An adorable romper! Belt and zipper were a lit...,14,420272,137lbs
1,36.0,straight & narrow,34b,gown,fit,"5' 6""",153475,10.0,other,"June 18, 2013",I felt so glamourous!!!,I rented this dress for a photo shoot. The the...,12,273551,132lbs
2,116.0,NaN,NaN,sheath,fit,"5' 4""",1063761,10.0,party,"December 14, 2015",It was a great time to celebrate the (almost) ...,This hugged in all the right places! It was a ...,4,360448,NaN
3,34.0,pear,34c,dress,fit,"5' 5""",126335,8.0,formal affair,"February 12, 2014",Dress arrived on time and in perfect condition.,I rented this for my company's black tie award...,8,909926,135lbs
4,27.0,athletic,34b,gown,fit,"5' 9""",616682,10.0,wedding,"September 26, 2016",Was in love with this dress !!!,I have always been petite in my upper body and...,12,151944,145lbs


In [0]:
remaing_df = rent_df.ix[:, ['user_id', 'item_id', 'rating','category']]
remaing_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 192544 entries, 0 to 192543
Data columns (total 4 columns):
user_id     192544 non-null int64
item_id     192544 non-null int64
rating      192462 non-null float64
category    192544 non-null object
dtypes: float64(1), int64(2), object(1)
memory usage: 7.3+ MB


/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  """Entry point for launching an IPython kernel.


In [0]:
missing_data = pd.DataFrame({'total_missing': remaing_df.isnull().sum(), 'perc_missing': (remaing_df.isnull().sum()/192544)*100})
missing_data

,perc_missing,total_missing
user_id,0.000000,0
item_id,0.000000,0
rating,0.042588,82
category,0.000000,0


In [0]:
remaing_df = remaing_df.drop_duplicates()
remaing_df = remaing_df.dropna()

In [0]:
missing_data = pd.DataFrame({'total_missing': remaing_df.isnull().sum(), 'perc_missing': (remaing_df.isnull().sum()/192544)*100})
missing_data

,perc_missing,total_missing
user_id,0.0,0
item_id,0.0,0
rating,0.0,0
category,0.0,0


In [0]:
remaing_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 192204 entries, 0 to 192543
Data columns (total 4 columns):
user_id     192204 non-null int64
item_id     192204 non-null int64
rating      192204 non-null float64
category    192204 non-null object
dtypes: float64(1), int64(2), object(1)
memory usage: 7.3+ MB


In [0]:
ratings = remaing_df.ix[:, ['user_id', 'item_id', 'rating']]
ratings = ratings.drop_duplicates()
ratings = ratings.dropna()
items = remaing_df.ix[:, ['item_id', 'category']]
items = items.drop_duplicates()
items = items.dropna()

/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  """Entry point for launching an IPython kernel.


In [0]:
ratings.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 192202 entries, 0 to 192543
Data columns (total 3 columns):
user_id    192202 non-null int64
item_id    192202 non-null int64
rating     192202 non-null float64
dtypes: float64(1), int64(2)
memory usage: 5.9 MB


In [0]:
items.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5855 entries, 0 to 192420
Data columns (total 2 columns):
item_id     5855 non-null int64
category    5855 non-null object
dtypes: int64(1), object(1)
memory usage: 137.2+ KB


In [0]:
ratings.to_csv('ratings.csv', index=0)
items.to_csv('items.csv', index=0)

In [0]:
# 安装pyspark
! pip install pyspark

In [0]:
# 构建RDD环境
from pyspark import SparkConf
from pyspark.context import SparkContext
sc = SparkContext.getOrCreate(SparkConf())

In [0]:
# 构建Spark SQL 环境
from pyspark.sql import SparkSession
from pyspark.sql import Row
spark = SparkSession \
        .builder \
        .appName("RDD_and_DataFrame") \
        .config("spark.some.config.option", "some-value") \
        .getOrCreate()

In [0]:
# read the ratings file
small_ratings_file = 'ratings.csv'
small_ratings_raw_data = sc.textFile(small_ratings_file)
small_ratings_raw_data_header = small_ratings_raw_data.take(1)[0]

In [0]:
small_ratings_raw_data.take(3)

[u'user_id,item_id,rating', u'420272,2260466,10.0', u'273551,153475,10.0']

In [0]:
small_ratings_raw_data_header

u'user_id,item_id,rating'

In [0]:
# now we can parse the raw data into a new RDD
small_ratings_data = small_ratings_raw_data.filter(lambda line: line!=small_ratings_raw_data_header)\
    .map(lambda line: line.split(",")).map(lambda tokens: (tokens[0],tokens[1],tokens[2])).cache()

In [0]:
# For illustrative purposes, we can take the first few lines of our RDD to see the result. 
small_ratings_data.take(3)

[(u'420272', u'2260466', u'10.0'),
 (u'273551', u'153475', u'10.0'),
 (u'360448', u'1063761', u'10.0')]

In [0]:
# we proceed in a similar way with the items.csv file

small_items_file = 'items.csv'
small_items_raw_data = sc.textFile(small_items_file)
small_items_raw_data_header = small_items_raw_data.take(1)[0]

small_items_data = small_items_raw_data.filter(lambda line: line!=small_items_raw_data_header)\
    .map(lambda line: line.split(",")).map(lambda tokens: (tokens[0],tokens[1])).cache()
    
small_items_data.take(3)

[(u'2260466', u'romper'), (u'153475', u'gown'), (u'1063761', u'sheath')]

In [0]:
# collaborative  filtering

# dataset:training data|validation data|testing data
training_RDD, validation_RDD, test_RDD = small_ratings_data.randomSplit([6, 2, 2], seed=0L)
validation_for_predict_RDD = validation_RDD.map(lambda x: (x[0], x[1]))
test_for_predict_RDD = test_RDD.map(lambda x: (x[0], x[1]))

In [0]:
# proceed with the training phase
from pyspark.mllib.recommendation import ALS
import math

In [0]:
seed = 5L
iterations = 10
regularization_parameter = 0.1
ranks = [16, 20, 24]
errors = [0, 0, 0]
err = 0
tolerance = 0.02

min_error = float('inf')
best_rank = -1
best_iteration = -1
for rank in ranks:
    model = ALS.train(training_RDD, rank, seed=seed, iterations=iterations,
                      lambda_=regularization_parameter)
    predictions = model.predictAll(validation_for_predict_RDD).map(lambda r: ((r[0], r[1]), r[2]))
    rates_and_preds = validation_RDD.map(lambda r: ((int(r[0]), int(r[1])), float(r[2]))).join(predictions)
    error = math.sqrt(rates_and_preds.map(lambda r: (r[1][0] - r[1][1])**2).mean())
    errors[err] = error
    err += 1
    print 'For rank %s the RMSE is %s' % (rank, error)
    if error < min_error:
        min_error = error
        best_rank = rank

print 'The best model was trained with rank %s' % best_rank

For rank 16 the RMSE is 6.49279681528
For rank 20 the RMSE is 6.33951908492
For rank 24 the RMSE is 6.06042819457
The best model was trained with rank 24


In [0]:
# let's explain this a little bit. First, let's have a look at how our predictions look
predictions.take(3)

[((403196, 2771965), 4.607870633144522),
 ((11852, 176025), -0.0995685872699319),
 ((559456, 1090064), 0.7626957780813342)]

In [0]:
rates_and_preds.take(3)

[((690593, 1273589), (8.0, 4.217940869966539)),
 ((981193, 702133), (10.0, 3.8700074004997047)),
 ((777582, 1729232), (8.0, 0.75765490940918))]

In [0]:
# To that, we apply a squared difference and the we use the mean() action to get the MSE and apply sqrt.
# finally, we test the selected model
model = ALS.train(training_RDD, best_rank, seed=seed, iterations=iterations,
                      lambda_=regularization_parameter)
predictions = model.predictAll(test_for_predict_RDD).map(lambda r: ((r[0], r[1]), r[2]))
rates_and_preds = test_RDD.map(lambda r: ((int(r[0]), int(r[1])), float(r[2]))).join(predictions)
error = math.sqrt(rates_and_preds.map(lambda r: (r[1][0] - r[1][1])**2).mean())
    
print 'For testing data the RMSE is %s' % (error)

For testing data the RMSE is 6.05975014603


In [0]:
# how to make recommendation
# So let's first load the movies complete file for later use.

complete_items_file = 'items.csv'
complete_items_raw_data = sc.textFile(complete_items_file)
complete_items_raw_data_header = complete_items_raw_data.take(1)[0]

# Parse
complete_items_data = complete_items_raw_data.filter(lambda line: line!=complete_items_raw_data_header)\
    .map(lambda line: line.split(",")).map(lambda tokens: (int(tokens[0]),tokens[1])).cache()

complete_items_titles = complete_items_data.map(lambda x: (int(x[0]),x[1]))
    
print "There are %s items in the complete dataset" % (complete_items_titles.count())

There are 5855 items in the complete dataset


In [0]:
complete_ratings_file = 'ratings.csv'
complete_ratings_raw_data = sc.textFile(complete_ratings_file)
complete_ratings_raw_data_header = complete_ratings_raw_data.take(1)[0]

# Parse
complete_ratings_data = complete_ratings_raw_data.filter(lambda line: line!=complete_ratings_raw_data_header)\
    .map(lambda line: line.split(",")).map(lambda tokens: (int(tokens[0]),tokens[1],tokens[2])).cache()
    
print "There are %s recommendations in the complete dataset" % (complete_ratings_data.count())
complete_ratings_data.take(3)

There are 192202 recommendations in the complete dataset


[(420272, u'2260466', u'10.0'),
 (273551, u'153475', u'10.0'),
 (360448, u'1063761', u'10.0')]

In [0]:
# Another thing we want to do, is give recommendations of items with a certain minimum number of ratings. 
# For that, we need to count the number of ratings per item.

def get_counts_and_averages(ID_and_ratings_tuple):
    nratings = len(ID_and_ratings_tuple[1])
    return ID_and_ratings_tuple[0], (nratings, float(sum(round(float(x)) for x in ID_and_ratings_tuple[1]))/nratings)

item_ID_with_ratings_RDD = (complete_ratings_data.map(lambda x: (x[1], x[2])).groupByKey())
item_ID_with_avg_ratings_RDD = item_ID_with_ratings_RDD.map(get_counts_and_averages)
item_rating_counts_RDD = item_ID_with_avg_ratings_RDD.map(lambda x: (x[0], x[1][0]))

In [0]:
item_ID_with_ratings_RDD.take(3)

[(u'2140811', <pyspark.resultiterable.ResultIterable at 0x7f40c594add0>),
 (u'2281848', <pyspark.resultiterable.ResultIterable at 0x7f40c594ac50>),
 (u'2703625', <pyspark.resultiterable.ResultIterable at 0x7f40c594a910>)]

In [0]:
item_ID_with_avg_ratings_RDD.take(3)

[(u'2140811', (18, 8.88888888888889)),
 (u'2281848', (50, 9.2)),
 (u'2703625', (35, 9.142857142857142))]

In [0]:
item_rating_counts_RDD.take(3)

[(u'2140811', 18), (u'2281848', 50), (u'2703625', 35)]

In [0]:
item_ID_with_avg_ratings_RDD.take(10)

[(u'2140811', (18, 8.88888888888889)),
 (u'2281848', (50, 9.2)),
 (u'2703625', (35, 9.142857142857142)),
 (u'2357523', (14, 8.857142857142858)),
 (u'1861964', (39, 9.076923076923077)),
 (u'472333', (47, 9.27659574468085)),
 (u'131117', (981, 9.033639143730888)),
 (u'322434', (52, 9.0)),
 (u'1532310', (3, 10.0)),
 (u'267405', (35, 9.6))]

In [0]:
# adding new user ratings
new_user_ID = 0

# The format of each line is (userID, itemID, rating)
new_user_ratings = [
     (0,2140811,9), 
     (0,2281848,9), 
     (0,2703625,9), 
     (0,2357523,8), 
     (0,1861964,9), 
     (0,472333,9), 
     (0,131117,9), 
     (0,322434,9), 
     (0,1532310,10) , 
     (0,267405,10) 
    ]

new_user_ratings_RDD = sc.parallelize(new_user_ratings)
print 'New user ratings: %s' % new_user_ratings_RDD.take(10)

New user ratings: [(0, 2140811, 9), (0, 2281848, 9), (0, 2703625, 9), (0, 2357523, 8), (0, 1861964, 9), (0, 472333, 9), (0, 131117, 9), (0, 322434, 9), (0, 1532310, 10), (0, 267405, 10)]


In [0]:
# Now we add them to the data we will use to train our recommender model. 
# We use Spark's union() transformation for this.
complete_data_with_new_ratings_RDD = complete_ratings_data.union(new_user_ratings_RDD)

In [0]:
# And finally we train the ALS model using all the parameters we selected before (when using the small dataset).
from time import time

t0 = time()
new_ratings_model = ALS.train(complete_data_with_new_ratings_RDD, best_rank, seed=seed, 
                              iterations=iterations, lambda_=regularization_parameter)
tt = time() - t0

print "New model trained in %s seconds" % round(tt,3)

New model trained in 18.634 seconds


In [0]:
# It took some time. We will need to repeat that every time a user add new ratings.
# Ideally we will do this in batches, and not for every single rating that comes into the system for every user.

In [0]:
# getting top recommendations
# Let's now get some recommendations!
# For that we will get an RDD with all the items the new user hasn't rated yet. 
# We will them together with the model to predict ratings.

new_user_ratings_ids = map(lambda x: x[1], new_user_ratings) # get just item IDs
# keep just those not on the ID list 
new_user_unrated_items_RDD = (complete_items_data.filter(lambda x: x[0] not in new_user_ratings_ids).map(lambda x: (new_user_ID, x[0])))

# Use the input RDD, new_user_unrated_items_RDD, with new_ratings_model.predictAll() to predict new ratings for the items
new_user_recommendations_RDD = new_ratings_model.predictAll(new_user_unrated_items_RDD)

In [0]:
new_user_ratings_ids

[2140811,
 2281848,
 2703625,
 2357523,
 1861964,
 472333,
 131117,
 322434,
 1532310,
 267405]

In [0]:
new_user_unrated_items_RDD.take(3)

[(0, 2260466), (0, 153475), (0, 1063761)]

In [0]:
new_user_unrated_items_RDD.count() # total have 5855items, not rating 5845

5845

In [0]:
new_user_recommendations_RDD.count()

5845

In [0]:
new_user_recommendations_RDD.take(10)

[Rating(user=0, product=1563416, rating=5.478723909414439),
 Rating(user=0, product=1099646, rating=11.10241579907407),
 Rating(user=0, product=713790, rating=7.382111315260923),
 Rating(user=0, product=2738558, rating=10.8274220956956),
 Rating(user=0, product=2048756, rating=9.62591778063876),
 Rating(user=0, product=1180202, rating=7.662109315005715),
 Rating(user=0, product=1300112, rating=10.667492279996337),
 Rating(user=0, product=2010548, rating=11.058191573758297),
 Rating(user=0, product=386314, rating=12.990129769619886),
 Rating(user=0, product=1348294, rating=8.448150933149781)]

In [0]:
# Transform new_user_recommendations_RDD into pairs of the form (item ID, Predicted Rating)
new_user_recommendations_rating_RDD = new_user_recommendations_RDD.map(lambda x: (x.product, x.rating))
new_user_recommendations_rating_title_and_count_RDD = \
    new_user_recommendations_rating_RDD.join(complete_items_titles).join(item_rating_counts_RDD.map(lambda x: (int(x[0]), x[1])))
new_user_recommendations_rating_title_and_count_RDD.take(3)

# 相当于是一个用户有很多没有没租过的衣服，根据模型计算出他会对每个衣服的评分，然后进行推荐前三个。

[(466944, ((4.123172974617845, u'gown'), 252)),
 (2107392, ((8.42363048717481, u'down'), 32)),
 (2576388, ((10.833968349772954, u'jumpsuit'), 8))]

In [0]:
# So we need to flat this down a bit in order to have (category, Rating, Ratings Count).
new_user_recommendations_rating_title_and_count_RDD = \
    new_user_recommendations_rating_title_and_count_RDD.map(lambda r: (r[0], r[1][0][1], r[1][0][0], r[1][1]))
new_user_recommendations_rating_title_and_count_RDD.take(3)

[(466944, u'gown', 4.123172974617845, 252),
 (2107392, u'down', 8.42363048717481, 32),
 (2576388, u'jumpsuit', 10.833968349772954, 8)]

In [0]:
# Finally, get the highest rated recommendations for the new user, filtering out items with less than 25 ratings.
top_items = new_user_recommendations_rating_title_and_count_RDD.filter(lambda r: r[3]>=25).takeOrdered(25, key=lambda x: -x[2])

print ('TOP recommended items (with more than 25 reviews):\n%s' %
        '\n'.join(map(str, top_items)))

TOP recommended items (with more than 25 reviews):
(1420896, u'gown', 14.184187605082625, 25)
(1606923, u'dress', 13.804242349538914, 31)
(2315001, u'jumpsuit', 13.729496587544082, 34)
(1584094, u'dress', 13.375911178826284, 43)
(1269149, u'gown', 13.316707606593885, 25)
(2931769, u'romper', 13.289865236963289, 34)
(1440124, u'dress', 13.035235093499773, 26)
(507766, u'dress', 13.021977989160643, 35)
(386314, u'sheath', 12.990129769619886, 37)
(2429745, u'romper', 12.962967679329056, 42)
(1207360, u'sheath', 12.952243789810975, 64)
(2635020, u'top', 12.94348618124923, 25)
(1069265, u'maxi', 12.865889015865037, 40)
(709521, u'dress', 12.863461735252645, 40)
(1155639, u'dress', 12.820026419765345, 29)
(1937688, u'dress', 12.811998256465639, 38)
(1519172, u'sheath', 12.797419319325227, 28)
(1846462, u'gown', 12.708789813709211, 25)
(1901762, u'dress', 12.703336415619257, 36)
(1944337, u'dress', 12.65806273008108, 39)
(1821947, u'shirtdress', 12.642846667047829, 37)
(1605839, u'dress', 12.

In [0]:
df_rating = spark.createDataFrame(complete_data_with_new_ratings_RDD)
df_rating.show(5)

+------+-------+----+
|    _1|     _2|  _3|
+------+-------+----+
|420272|2260466|10.0|
|273551| 153475|10.0|
|360448|1063761|10.0|
|909926| 126335| 8.0|
|151944| 616682|10.0|
+------+-------+----+
only showing top 5 rows



In [0]:
df_rating = df_rating.withColumnRenamed('_1', 'user_id').withColumnRenamed('_2', 'item_id').withColumnRenamed('_3', 'rating')
df_rating.show(5)

+-------+-------+------+
|user_id|item_id|rating|
+-------+-------+------+
| 420272|2260466|  10.0|
| 273551| 153475|  10.0|
| 360448|1063761|  10.0|
| 909926| 126335|   8.0|
| 151944| 616682|  10.0|
+-------+-------+------+
only showing top 5 rows



In [0]:
df_items = spark.createDataFrame(complete_items_data)
df_items = df_items.withColumnRenamed('_1', 'item_id').withColumnRenamed('_2', 'category')
df_items.show(5)

+-------+--------+
|item_id|category|
+-------+--------+
|2260466|  romper|
| 153475|    gown|
|1063761|  sheath|
| 126335|   dress|
| 616682|    gown|
+-------+--------+
only showing top 5 rows



In [0]:
df_rating.filter("user_id='0'").join(df_items, 'item_id').select('user_id', 'item_id', 'rating', 'category').show()

+-------+-------+------+--------+
|user_id|item_id|rating|category|
+-------+-------+------+--------+
|      0|2281848|     9|  jacket|
|      0|2703625|     9|  romper|
|      0| 131117|     9|    gown|
|      0| 322434|     9|    gown|
|      0| 472333|     9|   dress|
|      0|1861964|     9|   dress|
|      0|2357523|     8|  jacket|
|      0|2140811|     9|    vest|
|      0|1532310|    10|   dress|
|      0| 267405|    10|   dress|
+-------+-------+------+--------+



In [0]:
df_top_items = spark.createDataFrame(top_items)
df_top_items = df_top_items.withColumnRenamed('_1', 'item_id').withColumnRenamed('_2', 'category')\
.withColumnRenamed('_3', 'rating').withColumnRenamed('_4', 'num_review')
df_top_items.show()

+-------+--------+------------------+----------+
|item_id|category|            rating|num_review|
+-------+--------+------------------+----------+
|1420896|    gown|14.184187605082625|        25|
|1606923|   dress|13.804242349538914|        31|
|2315001|jumpsuit|13.729496587544082|        34|
|1584094|   dress|13.375911178826284|        43|
|1269149|    gown|13.316707606593885|        25|
|2931769|  romper|13.289865236963289|        34|
|1440124|   dress|13.035235093499773|        26|
| 507766|   dress|13.021977989160643|        35|
| 386314|  sheath|12.990129769619886|        37|
|2429745|  romper|12.962967679329056|        42|
|1207360|  sheath|12.952243789810975|        64|
|2635020|     top| 12.94348618124923|        25|
|1069265|    maxi|12.865889015865037|        40|
| 709521|   dress|12.863461735252645|        40|
|1155639|   dress|12.820026419765345|        29|
|1937688|   dress|12.811998256465639|        38|
|1519172|  sheath|12.797419319325227|        28|
|1846462|    gown|12